# Spreadsheet

    Make a spreadsheet using pinkfish.  This is useful for developing trading strategies.
    It can also be used as a tool for buy and sell signals that you then manually execute.

In [1]:
import datetime

import matplotlib.pyplot as plt
import pandas as pd

from talib.abstract import *

import pinkfish as pf
import pinkfish.itable as itable

# Format price data
pd.options.display.float_format = '{:0.2f}'.format

%matplotlib inline

In [2]:
# Set size of inline plots
'''note: rcParams can't be in same cell as import matplotlib
   or %matplotlib inline
   
   %matplotlib notebook: will lead to interactive plots embedded within
   the notebook, you can zoom and resize the figure
   
   %matplotlib inline: only draw static images in the notebook
'''
plt.rcParams["figure.figsize"] = (10, 7)

Some global data

In [3]:
symbol = 'SPY'
start = datetime.datetime(1900, 1, 1)
end = datetime.datetime.now()

Fetch symbol data from internet; do not use local cache. 

In [4]:
ts = pf.fetch_timeseries(symbol, use_cache=False)
ts.tail()

,high,low,open,close,volume,adj_close
date,,,,,,
2021-03-26,396.41,390.29,390.93,395.98,113023400.00,395.98
2021-03-29,396.75,392.81,394.40,395.78,108107600.00,395.78
2021-03-30,395.45,393.02,394.42,394.73,76262200.00,394.73
2021-03-31,398.00,395.31,395.34,396.33,112734200.00,396.33
2021-04-01,400.67,398.18,398.40,400.61,99599100.00,400.61


Select timeseries between start and end.  Back adjust prices relative to adj_close for dividends and splits.

In [5]:
ts = pf.select_tradeperiod(ts, start, end, use_adj=False)
ts.head()

,high,low,open,close,volume,adj_close
date,,,,,,
1993-01-29,43.97,43.75,43.97,43.94,1003200.00,25.88
1993-02-01,44.25,43.97,43.97,44.25,480500.00,26.07
1993-02-02,44.38,44.12,44.22,44.34,201300.00,26.12
1993-02-03,44.84,44.38,44.41,44.81,529400.00,26.40
1993-02-04,45.09,44.47,44.97,45.00,531500.00,26.51


Add techincal indicators

In [6]:
# Add 200 day MA.
ts['sma200'] = SMA(ts, timeperiod=200)

# Add ATR.
ts['atr'] = ATR(ts, timeperiod=14)

# Add 5 day high, and 5 day low
ts['high5'] = pd.Series(ts.high).rolling(window=5).max()
ts['low5'] = pd.Series(ts.low).rolling(window=5).min()

# Add RSI, and 2-period cumulative RSI
ts['rsi2'] = RSI(ts, timeperiod=2)
ts['c2rsi2'] = pd.Series(ts.rsi2).rolling(window=2).sum()

# Add midpoint
ts['mp'] = (ts.high + ts.low) / 2

# Add 10 day SMA of midpoint
ts['sma10'] = pd.Series(ts.mp).rolling(window=10).mean()

# Add temporary rolling 10 day Standard Deviation of midpoint
ts['__sd__'] = pd.Series(ts.mp).rolling(window=10).std()

# Add standard deviation envelope or channel around midpoint
ts['upper'] = ts.sma10 + ts['__sd__']*2
ts['lower'] = ts.sma10 - ts['__sd__']*2

# Drop temporary columns.
ts.drop(columns=['__sd__'], inplace=True)

Finalize timeseries

In [7]:
ts, start = pf.finalize_timeseries(ts, start)
ts.tail()

,high,low,open,close,volume,adj_close,sma200,atr,high5,low5,rsi2,c2rsi2,mp,sma10,upper,lower
date,,,,,,,,,,,,,,,,
2021-03-26,396.41,390.29,390.93,395.98,113023400.00,395.98,352.78,5.68,396.41,383.90,87.98,143.69,393.35,392.34,398.19,386.48
2021-03-29,396.75,392.81,394.40,395.78,108107600.00,395.78,353.26,5.56,396.75,383.90,84.08,172.05,394.78,392.38,398.30,386.46
2021-03-30,395.45,393.02,394.42,394.73,76262200.00,394.73,353.71,5.36,396.75,383.90,57.36,141.44,394.24,392.16,397.55,386.77
2021-03-31,398.00,395.31,395.34,396.33,112734200.00,396.33,354.16,5.21,398.00,383.90,78.34,135.70,396.65,392.25,397.94,386.56
2021-04-01,400.67,398.18,398.40,400.61,99599100.00,400.61,354.60,5.15,400.67,390.29,94.04,172.37,399.43,392.82,400.09,385.56


Select a smaller time from for use with itable

In [8]:
df = ts['2020-01-01':]
df.head()

,high,low,open,close,volume,adj_close,sma200,atr,high5,low5,rsi2,c2rsi2,mp,sma10,upper,lower
date,,,,,,,,,,,,,,,,
2020-01-02,324.89,322.53,323.54,324.87,59151200.00,317.87,296.66,2.00,324.89,320.15,88.53,141.38,323.71,321.54,323.95,319.13
2020-01-03,323.64,321.10,321.16,322.41,77709700.00,315.47,296.86,2.13,324.89,320.15,39.58,128.11,322.37,321.79,323.94,319.64
2020-01-06,323.73,320.36,320.49,323.64,55653900.00,316.67,297.06,2.22,324.89,320.15,61.09,100.67,322.04,321.97,323.83,320.11
2020-01-07,323.54,322.24,323.02,322.73,40496400.00,315.78,297.28,2.16,324.89,320.15,40.01,101.10,322.89,322.19,323.89,320.49
2020-01-08,325.78,322.67,322.94,324.45,68296000.00,317.46,297.50,2.23,325.78,320.36,73.97,113.98,324.23,322.48,324.49,320.46


Use itable to format the spreadsheet.  New 5 day high has blue highlight; new 5 day low has red highlight.

In [9]:
pt = itable.PrettyTable(
    df, tstyle=itable.TableStyle(theme='theme1'), center=True, header_row=True, rpt_header=20)

pt.update_col_header_style(
    format_function=lambda x: x.upper(), text_align='right')
pt.update_row_header_style(
    format_function=lambda x: pd.to_datetime(str(x)).strftime('%Y/%m/%d'), text_align='right')

for col in range(pt.num_cols):
    if pt.df.columns[col] == 'volume':
        pt.update_cell_style(cols=[col], format_function=lambda x: format(x, '.0f'), text_align='right')
    else:
        pt.update_cell_style(cols=[col], format_function=lambda x: format(x, '.2f'), text_align='right')

for row in range(pt.num_rows):
    if row == 0:
        continue
    if (pt.df['high5'][row] == pt.df['high'][row]) and \
       (pt.df['high5'][row] > pt.df['high'][row-1]):
        col = df.columns.get_loc('high5')    
        pt.update_cell_style(rows=[row], cols=[col], color='blue')
    if (pt.df['low5'][row] == pt.df['low'][row]) and \
       (pt.df['low5'][row] < pt.df['low'][row-1]):
        col = df.columns.get_loc('low5')
        pt.update_cell_style(rows=[row], cols=[col], color='maroon')          

In [10]:
pt

,HIGH,LOW,OPEN,CLOSE,VOLUME,ADJ_CLOSE,SMA200,ATR,HIGH5,LOW5,RSI2,C2RSI2,MP,SMA10,UPPER,LOWER
2020/01/02,324.89,322.53,323.54,324.87,59151200,317.87,296.66,2.00,324.89,320.15,88.53,141.38,323.71,321.54,323.95,319.13
2020/01/03,323.64,321.10,321.16,322.41,77709700,315.47,296.86,2.13,324.89,320.15,39.58,128.11,322.37,321.79,323.94,319.64
2020/01/06,323.73,320.36,320.49,323.64,55653900,316.67,297.06,2.22,324.89,320.15,61.09,100.67,322.04,321.97,323.83,320.11
2020/01/07,323.54,322.24,323.02,322.73,40496400,315.78,297.28,2.16,324.89,320.15,40.01,101.10,322.89,322.19,323.89,320.49
2020/01/08,325.78,322.67,322.94,324.45,68296000,317.46,297.50,2.23,325.78,320.36,73.97,113.98,324.23,322.48,324.49,320.46
2020/01/09,326.73,325.52,326.16,326.65,48473300,319.61,297.73,2.23,326.73,320.36,89.37,163.33,326.12,322.97,325.83,320.11
2020/01/10,327.46,325.20,327.29,325.71,53029300,318.70,297.96,2.23,327.46,320.36,59.36,148.73,326.33,323.37,326.88,319.86
2020/01/13,327.96,325.92,326.39,327.95,47086800,320.89,298.20,2.24,327.96,322.24,84.37,143.73,326.94,323.76,327.91,319.61
2020/01/14,328.62,326.84,327.47,327.45,62832800,320.40,298.42,2.20,328.62,322.67,66.19,150.56,327.73,324.35,328.94,319.77
2020/01/15,329.02,327.26,327.35,328.19,72056600,321.12,298.63,2.17,329.02,325.20,79.36,145.54,328.14,325.05,329.59,320.51
